# Introduction to Quantum Error

In [ ]:
import numpy as np

## Contents
1. [Introduction](#introduction)
2. [Describing Quantum states and gates](#basics)
3. [Quantum Errors: Cause and Effect](#errors)
    1. [Coherent Quantum Errors: Gates which are incorrectly applied](#coherent)
    2. [Environmental Decoherence](#decoherence)
        1. [Density Matrix representation](#density)
        2. [Partial Trace Example](#partial)
    4. [Simple models of loss, leakage, measurement and initialization](#models)
        1. [Determine Measurement projectors](#projectors)
        2. [Determine Probabilities of Collapsed State](#probabilities)
        3. [Extra Information](#extra)

## 1. Introduction  <a id='introduction'></a>

In this file we will go through the [Devitt el al. paper (arXiv: 0905.2794v4)](https://arxiv.org/abs/0905.2794v4) Sections ll and lll and derive and explain all of the math and theory to fully understand basic quantum errors.

## 2. Describing Quantum states and gates <a id='basics'></a>

This section describes:
    (a) The basic structure of a qubit
    (b) Some general requirements of QEC

Suggest reading this section of the paper to understand the fundamentals of qubits and why we need QEC

We can describe the state of a qubit by $\vert\psi\rangle = \alpha\vert0\rangle + \beta\vert1\rangle$ where $\vert0\rangle$ and $\vert1\rangle$ are 2 orthonormal basis states known as the computational basis, and we know that  $\vert\alpha\vert^2 + \vert\beta\vert^2 = 1$ holds. 

The probabilty of being in the $\vert0\rangle$ state is $P(\vert0\rangle) = \vert\alpha\vert^2$ and for the $\vert1\rangle$ state is $P(\vert1\rangle) = \vert\beta\vert^2$.

Note that we can also define other orthonormal basis states like $\vert+\rangle$ and $\vert-\rangle$ where $\vert+\rangle = \frac{1}{\sqrt2}(\vert0\rangle + \vert1\rangle)$ and $\vert-\rangle = \frac{1}{\sqrt2}(\vert0\rangle - \vert1\rangle)$


Many Quantum gates can be represented using the Pauli operators $(\sigma_x, \sigma_y, \sigma_z, \sigma_I)$ below. These are important gates in quantum computation.


In [ ]:
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,1j],[-1j,0]])
sigma_z = np.array([[1,0],[0,-1]])
sigma_I = np.identity(2)
print('Below we show the Pauli Operators which can be used to describe any quantum gate on an individual qubit:')
print('Pauli-X: \n', sigma_x) 
print('Pauli-Y: \n', sigma_y)
print('Pauli-Z: \n', sigma_z)
print('Pauli-I: \n', sigma_I)

## 3. Quantum Errors: Cause and Effect <a id='errors'></a>

In this section we consider different sources of error -- coherent, environmental decoherence, and loss, leakage, measuremnt and initialization.
We will apply these errors to two different algorithms.

1. A single qubit undergoing N identity opertations
$$\vert\psi_{final}\rangle = \prod_{i}^{N} I_{i} \vert0\rangle = \vert0\rangle $$
where $I \equiv \sigma_{I}$

If a qubit is initially in the $\vert0\rangle $state then this operation in the basis $\vert0\rangle$, $\vert1\rangle$ would be $\vert0\rangle$
This can be shown with the code below. (Feel free to play with the initial state values and see what results :) )

In [ ]:
initial_state = np.array([1, 0]) # initalize initial state to |0>
n = 10 # number of times to apply the I operator (can be anything)
I_i = np.identity(2) # initialize I_i for i = 1 first

# Since we are only indexing over i we can multiply I_i with sigma_I n times to arrive at I_n final
I_n = np.dot(I_i, sigma_I)
for i in range(1, n):
    I_n = np.dot(I_n, sigma_I)

final_state = np.dot(I_n, initial_state)
print('Here we can see that the final state is the same as the initial state. ')
print('Final State: ', final_state)

2. An algorithm of 3 gates acting on a qubit

The Hadamard Gate: $ H = \frac{1}{\sqrt2} \begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix}$
$$\vert\psi\rangle = HIH\vert0\rangle = HI\frac{1}{\sqrt{2}}(\vert0\rangle + \vert1\rangle) = H\frac{1}{\sqrt{2}}(\vert0\rangle + \vert1\rangle) = \vert0\rangle$$

We can show the above case is correct by applying this algorithm to a single qubit in the $\vert0\rangle$ state. (Feel free to play with the initial state values and see what results :))

In [ ]:
initial_state = np.array([1,0]) # initalize initial state to |0>
h = 1/np.sqrt(2)*np.array([[1,1],[1,-1]]) # set the hadamard operator 

# Below we perform the 3 gate operation and psi just represents the state of the qubit during the operations
psi = np.dot(h, initial_state) # apply the first hadamard gate
psi = np.dot(sigma_I, psi) # apply a wait stage with the I gate
final_state = np.dot(h, psi) # apply the second hadamard gate

print('Here we can see that the final state is: ')
print('Final State: ', (np.rint(final_state)).astype(int))

### A. Coherent Quantum Errors: Gates which are incorrectly applied <a id='coherent'></a>

This error is usually related to incorrect knowledge of the system dynamics, but it is coherent

First we will consider an example which applies an incorrect rotation about the X-axis of the Bloch Sphere:

$$ \vert\psi_{final}\rangle = \prod_{k}^{N} e^{i\epsilon\sigma_{x}}\vert0\rangle = \cos(N\epsilon)\vert0\rangle + i\sin(N\epsilon)\vert1\rangle$$

Now we will measure the system in the $\vert0\rangle$, $\vert1\rangle$ basis. Due to the coherent quantum errors below is the probability of measuing the system in $\vert0\rangle$ or $\vert1\rangle$:

$$ P(\vert0\rangle) = \cos^2(N\epsilon) \approx 1 - (N\epsilon)^2 $$
$$ P(\vert1\rangle) = \sin^2(N\epsilon) \approx (N\epsilon)^2$$

Thus the probability of error is given below when we want the 'correct' state to be $\vert0\rangle$:

$$ P_{error} \approx (N\epsilon)^2$$ 

Below we can see a coherent quantum error acting on the initial state with arbitrary $\epsilon$ and calculate the error due to the X-rotation error N times.

In [ ]:
initial_state = np.array([1,0]) # initalize initial state to |0>
epsilon = 1e-3 # arbitrary epsilon is set
n = 10 # arbitrary N is set
# We can convert the exponential to its trigonometric form using euler's identity and some algebraic manipulation
final_state = np.dot(np.cos(n*epsilon), initial_state) + np.dot(1j*np.sin(n*epsilon), np.array([0,1]))
print('Final State: ', final_state)

We know that the probability of being in a certain state say $\vert0\rangle$ is $P(\vert0\rangle) = \alpha^2$ and for $\vert1\rangle$ is $P(\vert1\rangle) = \beta^2$ when $\vert\psi\rangle = \alpha\vert0\rangle + \beta\vert1\rangle$

In [ ]:
prob_zero = (final_state[0].real**2 + final_state[0].imag**2) # Probability of being in the |0> state
prob_one = (final_state[1].real**2 + final_state[1].imag**2) # Probability of being in the |1> state

print('Probability of |0> State: ', prob_zero, '\nProbability of |1> State: ', prob_one)
print('We can see when adding the two probabilities we get one: ', (np.rint(prob_zero + prob_one).real).astype(int))

In [ ]:
#### Calculating the probability for error: (This is the same as the P(|1>) or 1 - P(|0>))
prob_error = prob_one
print('Probability of error: prob_one = ', prob_error, ' = 1 - prob_zero = ', 1 - prob_zero)

### B. Environmental Decoherence <a id='decoherence'></a>

This type of error usually stems from noise, heat, or other external factors.
In this example we will focus on an environment which is a two level system and has two basis states: $\vert e_0\rangle$ and $ \vert e_1\rangle$.
These satisfy the relations $ \langle e_i\vert e_j\rangle = \delta_{ij}$ and $ \vert e_0\rangle\langle e_0\vert + \vert e_1\rangle\langle e_1\vert = I$.

We will assume 2 things:

1. The environment coupling works as follows: when the qubit is in the $\vert1\rangle$ state the coupling flips the environment state, and when the qubit is in the $\vert0\rangle$ state nothing happens.
2. The system/environemnt only interact during the wait stage when the identity operator is acting.

Let us assume the environment is initialized to the state $\vert E\rangle = \vert e_0\rangle$ and lets couple it to the 2nd system we considered in part lll which applies 3 gates on the initial state:
$$ HIH\vert0\rangle\vert E\rangle = \frac{1}{2}(\vert0\rangle + \vert1\rangle)\vert e_0\rangle + \frac{1}{2}(\vert0\rangle - \vert1\rangle)\vert e_1\rangle$$

This can be simplified to the following since the error operator flips when acting on $\vert1\rangle$:
$$ HIH\vert0\rangle\vert E\rangle = HI\frac{1}{\sqrt{2}}(\vert0\rangle + \vert1\rangle)\vert e_0\rangle 
= H\frac{1}{\sqrt{2}}(\vert0\rangle\vert e_0\rangle + \vert1\rangle\vert e_1\rangle)
=\frac{1}{2}(\vert0\rangle + \vert1\rangle)\vert e_0\rangle + \frac{1}{2}(\vert0\rangle - \vert1\rangle)\vert e_1\rangle$$

Below is a representation to implement this error operation as a multiplication of matrices:
$$(H_q \otimes I_E)E_A(I_q \otimes I_E)(H_q \otimes I_E)\vert qE_0\rangle$$
where $$H_q =\frac{1}{\sqrt2} \begin{pmatrix}
1 & 1 \\
1 & -1
\end{pmatrix},$$

$$I_E = I_q =\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix},$$

$$E_A = \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0
\end{pmatrix}$$

$E_A$ is similar to the CNOT gate in this case due to the assumptions made by the paper.

The final state below is exactly the state we get when applying the $HIH\vert0\rangle\vert E\rangle$ operation.

In [ ]:
error_gate = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]) # E_A
hadamard = np.kron(h, sigma_I) # H_q
identity = np.kron(sigma_I, sigma_I) # I_E = I_q
initial_state = np.array([1, 0, 0, 0]) # Tensor of initial state of qubit |0> and E = |e_0>
final_state = np.dot(hadamard, np.dot(error_gate, np.dot(identity, np.dot(hadamard, initial_state))))
print('Final State: ', final_state)

### a. Density Matrix representation  <a id='density'></a>

However, now since we are considering decoherent operations, we are no longer in a pure state. Instead we have turned our quantum states into a classical mixed state, and we must move to a density matrix to be able to represent it. This matrix represents the lack of knowledge of the system, being a description of the system at all possible measurement results. 

We know that the density matix for a given state is $ \rho_f = \sum_{j}^{}{p_j\vert\psi_j\rangle\langle\psi_j\vert}$ where $p_j$ is the probability of being in the state $\vert\psi_j\rangle$. 

To take the density matrix of the state $\vert\psi\rangle = \frac{1}{2}(\vert0\rangle + \vert1\rangle)\vert e_0\rangle + \frac{1}{2}(\vert0\rangle - \vert1\rangle)\vert e_1\rangle$ from above:

$$ \rho_f = \sum_{j}^{}{p_j\vert\psi_j\rangle\langle\psi_j\vert} = p\vert\psi\rangle\langle\psi\vert = \biggl(\frac{1}{2}\biggr)^2 \biggl((\vert0\rangle + \vert1\rangle)\vert e_0\rangle + (\vert0\rangle - \vert1\rangle)\vert e_1\rangle\biggr) \biggl((\langle0\vert + \langle1\vert)\langle e_0\vert + (\langle0\vert - \langle1\vert)\langle e_1\vert \biggr) $$ 

$$ = \frac{1}{4} \biggl[ \biggl( (\vert0\rangle + \vert1\rangle)\vert e_0\rangle\biggr)
\biggl( (\langle0\vert + \langle1\vert)\langle e_0\vert\biggr) + \biggl( (\vert0\rangle - \vert1\rangle)\vert e_1\rangle\biggr) \biggl( (\langle0\vert - \langle1\vert)\langle e_1\vert\biggr) \\+ \biggl( (\vert0\rangle + \vert1\rangle)\vert e_0\rangle\biggr)
\biggl( (\langle0\vert - \langle1\vert)\langle e_1\vert\biggr) + \biggl( (\vert0\rangle - \vert1\rangle)\vert e_1\rangle\biggr) \biggl( (\langle0\vert + \langle1\vert)\langle e_0\vert\biggr) \biggr]$$


After a little bit of algebra from above we arrive at:
$$ \rho_f = \frac{1}{4}(\vert0\rangle\langle0\vert + \vert0\rangle\langle1\vert + \vert1\rangle\langle0\vert + \vert1\rangle\langle1\vert)\vert e_0\rangle\langle e_0\vert
+ \frac{1}{4}(\vert0\rangle\langle0\vert - \vert0\rangle\langle1\vert - \vert1\rangle\langle0\vert + \vert1\rangle\langle1\vert)\vert e_1\rangle\langle e_1\vert$$
$$ + \frac{1}{4}(\vert0\rangle\langle0\vert - \vert0\rangle\langle1\vert + \vert1\rangle\langle0\vert - \vert1\rangle\langle1\vert)\vert e_0\rangle\langle e_1\vert
+ \frac{1}{4}(\vert0\rangle\langle0\vert + \vert0\rangle\langle1\vert - \vert1\rangle\langle0\vert - \vert1\rangle\langle1\vert)\vert e_1\rangle\langle e_0\vert $$

Since we dont know the environmental degrees of freedom we will trace over this part of the system (also known as a partial trace): 

https://en.wikipedia.org/wiki/Partial_trace#Partial_trace_for_operators_on_Hilbert_spaces),

https://www.ryanlarose.com/uploads/1/1/5/8/115879647/quic06-states-trace.pdf (pg6),

http://mmrc.amss.cas.cn/tlb/201702/W020170224608149940643.pdf (Section 2.4.3)

$$ TR_E (\rho_f) = \frac{1}{4}(\vert0\rangle\langle0\vert + \vert0\rangle\langle1\vert + \vert1\rangle\langle0\vert + \vert1\rangle\langle1\vert)
+ \frac{1}{4}(\vert0\rangle\langle0\vert - \vert0\rangle\langle1\vert - \vert1\rangle\langle0\vert + \vert1\rangle\langle1\vert) = \frac{1}{2} \biggl( \vert0\rangle\langle0\vert+ \vert1\rangle\langle1\vert \biggr) $$

As we can see from this result, the system is $\vert0\rangle$ 50% of the time and $\vert1\rangle$ 50% of the time. Additionally, the 2nd hadamard gate has no effect on the qubit state since this sequence of gates should take the state $ \frac{1}{2}(\vert0\rangle + \vert1\rangle) $ to $ \vert0\rangle$ but does not due to the environmental coupling.

### b. Partial Trace Example <a id='partial'></a>

Let $\rho_{AB} = \vert\Phi_{AB}^+\rangle\langle\Phi_{AB}^+\vert$,  where $ \vert\Phi_{AB}^+\rangle = \frac{1}{\sqrt2}\biggl(\vert00\rangle + \vert11\rangle\biggr)$. Therefore the density operator is 

$$ \rho_{AB} = \frac{1}{2}\biggl(\vert00\rangle\langle00\vert + \vert00\rangle\langle11\vert + \vert11\rangle\langle00\vert + \vert11\rangle\langle11\vert\biggr)$$

Now the reduced density operator (Partial Trace over B) for system A defined as $\rho_A = TR_B(\rho_{AB})$. Thus since $TR_B$ acts on the B system we have $TR_B(\vert a_1\rangle\langle a_2\vert \otimes \vert b_1\rangle\langle b_2\vert) =  \vert a_1\rangle\langle a_2\vert TR(\vert b_1\rangle\langle b_2\vert)$.
And finally since $TR(\vert b_1\rangle\langle b_2\vert) = \langle b_2\vert b_1\rangle $ it is clear to see that $ TR_B(\rho_{AB}) = \vert a_1\rangle\langle a_2\vert \langle b_2\vert b_1\rangle $.

Thus for this example: 

$$TR_B(\rho_{AB}) = \frac{1}{2} \biggl( TR_B (\vert00\rangle\langle00\vert) + TR_B (\vert00\rangle\langle11\vert) + TR_B (\vert11\rangle\langle00\vert) + TR_B (\vert11\rangle\langle11\vert)\biggr)$$

$$ = \frac{1}{2} \biggl( \vert0\rangle\langle0\vert \langle0\vert0\rangle + \vert0\rangle\langle1\vert \langle1\vert0\rangle + \vert1\rangle\langle0\vert \langle0\vert1\rangle + \vert1\rangle\langle1\vert \langle1\vert1\rangle \biggr) = \frac{1}{2} \biggl( \vert0\rangle\langle0\vert \cdot (1) + \vert0\rangle\langle1\vert \cdot (0) + \vert1\rangle\langle0\vert \cdot (0) + \vert1\rangle\langle1\vert \cdot (1) \biggr) =  \frac{1}{2} \biggl( \vert0\rangle\langle0\vert+ \vert1\rangle\langle1\vert \biggr)$$

## C. Simple models of loss, leakage, measurement and initialization <a id='models'></a>

In this section, important & frequently used models will be discerned to identify Quantum Errors.

First we will model Measurement Errors using 2 methods.

#### Positive Operator Value Measures (POVM's)

POVMs in quantum error correction help us understand and deal with errors by allowing measurements that account for variations from the intended quantum states.
In Method 1 we use the following POVM's: 

$$F_0 = \left ( 1-p_m \right ) |0\rangle\langle0| + p_m |1\rangle\langle1|$$
$$F_1 = \left ( 1-p_m \right ) |1\rangle\langle1| + p_m |0\rangle\langle0|$$

* $F_0 (F_1)$: This represents the operator corresponding to the outcome $\vert0\rangle$ ($\vert1\rangle$) respectively in the measurement. 
* $p_m$: This variable represents the probability of measurement error, in this case it would be the probability of obtaining the outcome $\vert1\rangle$ ($\vert0\rangle$) respectively (as opposed to outcome $\vert0\rangle$ ( $\vert1\rangle$) respectively) in the measurement. It is a parameter that can be adjusted to control the likelihood of different outcomes.
* $(1 - p_m)$: This term represents the probability of obtaining outcome $\vert0\rangle$ ($\vert1\rangle$) respectivelyin the measurement. It is derived from subtracting the probability of outcome $\vert1\rangle$ ($\vert0\rangle$) respectively from 1, ensuring that the probabilities sum up to 1.

In [ ]:
# Compute density matrix (rho)
psi = np.array([[1, 0]])
rho = np.kron(psi, psi.conjugate().T)
print(rho)
# Computing  F_0 and F_1

# Define the parameter p_m (probability of outcome "1")
p_m = 0.3  # You can change this value as desired

# Define the column vectors for |0> and |1>
ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])

# Define the row vectors for <0| and <1|
bra_0 = np.array([[1],[0]])
bra_1 = np.array([[0],[1]])

# Finally, we can compute F_0 using np.kron for both ket and bra vectors
# Since |0> <0|, or ket-0 (2x1 vector) x bra-0 (1x2 scalar)
F_0 = (1 - p_m) * np.kron(ket_0, bra_0) + p_m * np.kron(ket_1, bra_1)
print("F_0:\n", F_0)

# Compute F_1 using np.kron for both ket and bra vectors
F_1 = (1 - p_m) * np.kron(ket_1, bra_1) + p_m * np.kron(ket_0, bra_0)
print("F_1:\n", F_1)

#### In Method 2 we apply the following mapping to the qubit:

$$ \rho \to \rho ' = \left ( 1-p_m \right )\rho + p_m X\rho X $$

Applying the mapping to a qubit means transforming the state of the qubit according to the given formula to account for the effect of a certain operation or measurement. So, the meaning of the mapping is that it modifies the original state of the qubit to account for the specified scaling and transformation operations.  Therefore, the mapping onto the qubit transforms the initial state p according, where p' is the updated state of the qubit.
$\rho$ ' = updated state of qubit

Below we show an example of how we would compute $\rho'$:

$$\rho' = (1 - p_m) \rho + p_m X\rho X = (1 - p_m) \begin{pmatrix} \rho_{00} & \rho_{01} \\ \rho_{10} & \rho_{11} \end{pmatrix} + p_m \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \begin{pmatrix} \rho_{00} & \rho_{01} \\ \rho_{10} & \rho_{11} \end{pmatrix} \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} = (1 - p_m) \begin{pmatrix} \rho_{00} & \rho_{01} \\ \rho_{10} & \rho_{11} \end{pmatrix} + p_m \begin{pmatrix} \rho_{01} & \rho_{00} \\ \rho_{11} & \rho_{10} \end{pmatrix}$$

Hence, the expression for the updated state $\rho'$ after applying the mapping equation is:

$$\rho' = \begin{pmatrix} (1 - p_m) \rho_{00} + p_m \rho_{01} & (1 - p_m) \rho_{01} + p_M \rho_{00} \\ (1 - p_m) \rho_{10} + p_m \rho_{11} & (1 - p_m) \rho_{11} + p_m \rho_{10} \end{pmatrix}$$

This derivation shows how the original state $\rho$ is transformed to the updated state $\rho'$ by scaling the elements of $\rho$ with $(1 - p_m)$ and performing the X-gate transformation on the corresponding elements.

In [ ]:
# Define the initial state of the qubit, rho
print("rho", rho)

# Compute rho' using the given formula
rho_prime =  (1-p_m)*rho + p_m*np.dot(sigma_x, np.dot(rho, sigma_x))
print("rho prime (updated)", rho_prime)


## a. Determine Measurement projectors <a id='projectors'></a>

We will first define the measurement projectors $A_0$ and $A_1$ as

$$A_0 = |0\rangle\langle0| \space and \space A_1 = |1\rangle\langle1|$$

When we perform a measurement, we are interested in obtaining information about the state of a system. A measurement projector (like $A_0$ and $A_1$ helps us extract that info by projecting the state onto the specific outcome we want to observe. In this case, we want to project the measurements onto the $(|0\rangle, |1\rangle)$ basis.

Tracing $F_0$ and $F_1$ with the quantum state $\rho$ gives us insights into the probabilities and expected values of the measurement outcomes, helping us understand the behavior of the quantum system.
After performing a perfect measurement in the $(|0\rangle, |1\rangle)$ basis, we can calculate the trace of $F_0$ operating on the quantum state $\rho$.

In quantum mechanics, the POVM element $F_i$ is associated with the measurement outcome i, such that the probability of obtaining it when making a measurement on the quantum state $\rho$ is given by $Prob(i)=tr(\rho F_i)$

#### Using Method 1 ($\rho$):

$$Tr(F_0 \rho)=\left( 1-p_M \right)Tr\left( A_0 \rho \right))+ p_MTr(A_1\rho)),$$
$$Tr(F_1 \rho)=\left( 1-p_M \right)Tr\left( A_1 \rho \right))+ p_MTr(A_0\rho)),$$

#### Using Method 2 ($\rho'$):

$$Tr(A_0 \rho')=\left( 1-p_M \right)Tr\left(A_0 \rho \right))+ p_MTr(XA_0 X_p))$$
$$= \left( 1-p_M \right)Tr\left(A_0 \rho \right))+ p_MTr(A_1\rho))$$
$$$$
$$Tr(A_1 \rho')=\left( 1-p_M \right)Tr\left(A_1 p \right))+ p_MTr(XA_1 X_p))$$
$$= \left( 1-p_M \right)Tr\left(A_1 \rho \right))+ p_MTr(A_0\rho))$$


## b. Determine Probabilities of Collapsed State  <a id='probabilities'></a>

Now we will find probabilties of being in a certain state by tracing

#### Introduction to Tracing

1. "Trace" refers to a math operation performed on operators or matrices (as you'll say below)
2. The trace of a matrix is obtained by summing its diagonal elements.
3. When analyzing measurements, we use tracing to evaluate the probabilities of specific measurement outcomes.
4. Since it's a math operation, it's not directly related to manipulating the physical qubits themselves.
5. Finally, tracing helps us learn about the presence or absence of specific quantum states by evaluating the diagonal elements of operators or matrices.

See [Wikipedia](https://en.wikipedia.org/wiki/POVM) for more on POVMs. We will go over an example of deriving $\text{Tr}(A_0 \rho')$ below. To derive the equation, let's start with the expression:

$$\text{Tr}(A_0 \rho') = \left(1-p_M\right)\text{Tr}(A_0 \rho) + p_M\text{Tr}(XA_0 X\rho)$$

First, let's calculate $\text{Tr}(XA_0 X\rho)$:

$$\text{Tr}(A_0 \rho') = \left(1-p_M\right)\text{Tr}\left(\begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix} \begin{pmatrix} \rho_{00} & \rho_{01} \\ \rho_{10} & \rho_{11} \end{pmatrix}\right) + p_M\text{Tr}\left(\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \cdot \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix} \cdot \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \cdot \begin{pmatrix} \rho_{00} & \rho_{01} \\ \rho_{10} & \rho_{11} \end{pmatrix} \right)$$

Simplifying the matrix multiplication and take the trace of each:

$$ = \left(1-p_M\right)\text{Tr}\begin{pmatrix} \rho_{00} & \rho_{01} \\ 0 & 0 \end{pmatrix} + p_M\text{Tr}\begin{pmatrix} 0 & 0 \\ \rho_{10} & \rho_{11} \end{pmatrix} = \left(1-p_M\right) \rho_{00} + p_M \rho_{11}$$

In [ ]:
def trace_rho(rho, rho_prime):
    #This function uses both rho and rho' for the two methods involved in tracing


    # Define the measurement projectors A_0 and A_1
    A_0 = np.kron(ket_0, bra_0)  # Measurement projector for |0>
    A_1 = np.kron(ket_1, bra_1)  # Measurement projector for |1>

    # Calculate the trace of F_0 multiplied by the quantum state p
    traceF0_rho = np.trace(np.dot(F_0, rho))

    # Calculate the trace of F_1 multiplied by the quantum state p
    traceF1_rho = np.trace(np.dot(F_1, rho))

    # Calculate the trace of A_0 multiplied by the updated quantum state p_prime
    traceA0_rho_prime = np.trace(np.dot(A_0, rho_prime))

    # Calculate the trace of A_1 multiplied by the updated quantum state p_prime
    traceA1_rho_prime = np.trace(np.dot(A_1, rho_prime))

    return traceF0_rho, traceF1_rho, traceA0_rho_prime, traceA1_rho_prime
    # Print the measurement outcome probabilities

So, either method will result in the same probabilities 

### Qubit Collapse

When using the Positive Operator Value Measures (POVMs) (Method 1), the collapsed state of the qubit is given by:

$$\rho\to \frac {M_i\rho M_i^{\dagger}}{Tr\left( F_i\rho \right)},\quad i=0,1,$$

where we use the measurement operators below:

$$M_0=\sqrt{1-p_M} |0\rangle\langle0| + \sqrt{p_M}|1\rangle\langle1|,$$
$$M_1=\sqrt{1-p_M} |1\rangle\langle1| + \sqrt{p_M}|0\rangle\langle0|$$

Below is a representation for the computation using  i=0 for $ \rho \to$ $\frac{M_0 \rho M_0^\dagger}{\text{Tr}(F_0 \rho)}$


First, calculate the terms for

$$M_0=\sqrt{1-p_M} |0\rangle\langle0| + \sqrt{p_M}|1\rangle\langle1| = \sqrt{1-p_M} \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix} + \sqrt{p_M} \begin{pmatrix} 0 & 0 \\ 0 & 1 \end{pmatrix} = \begin{pmatrix} \sqrt{1-p_M} & 0 \\ 0 & \sqrt{p_M} \end{pmatrix}$$


Plugging into for 

$$ \rho \to \frac{M_0 \rho M_0^\dagger}{\text{Tr}(F_0 \rho)} \begin{pmatrix} \sqrt{1-p_M} & 0 \\ 0 & \sqrt{p_M} \end{pmatrix} \rho\begin{pmatrix} \sqrt{1-p_M} & 0 \\ 0 & \sqrt{p_M} \end{pmatrix} ^\dagger$$

Then dividing by our Trace where $F_0 \rho = \left ( 1-p_m \right ) \vert0\rangle\langle0\vert + p_m \vert1\rangle\langle1\vert  \rho$ and  $Tr(F_0 \rho)=\left( 1-p_M \right)Tr\left( A_0 \rho \right))+ p_MTr(A_1\rho))$

Substituting the values of $A_0$ and $A_1$ from $F_0$ and $F_1$: 

$$ Tr(F_0 \rho) = (1-p_M)Tr\left(\begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix} \rho \right) + p_MTr\left(\begin{pmatrix} 0 & 0 \\ 0 & 1 \end{pmatrix} \rho \right) = (1-p_M)Tr\left(\begin{pmatrix} \rho_{00} & 0 \\ 0 & 0 \end{pmatrix} \right) + p_MTr\left(\begin{pmatrix} 0 & 0 \\ 0 & \rho_{11} \end{pmatrix} \right)$$

Since the trace of a 2x2 matrix is simply the sum of its diagonal elements:

$$ Tr(F_0 \rho) = (1-p_M)(\rho_{00} + \rho_{11}) + p_M \rho_{11} $$

Now, we can substitute this expression back into the original expression:

$$\frac{M_0 \rho M_0^\dagger}{Tr(F_0 \rho)} = \frac{\begin{pmatrix} \sqrt{1-p_M} & 0 \\ 0 & \sqrt{p_M} \end{pmatrix} \rho \begin{pmatrix} \sqrt{1-p_M} & 0 \\ 0 & \sqrt{p_M} \end{pmatrix}^\dagger}{(1-p_M)\rho_{00} + p_M \rho_{11}} $$

Below we make a function that calculates a density matrix from a given $\vert\psi\rangle$:

In [ ]:
def compute_density_matrix(psi):
    psi = np.array(psi)
    psi_conj = np.conjugate(psi)
    num_states = len(psi)  # Number of states in the superposition
    rho_f = np.zeros((num_states, num_states), dtype=complex)  # Initialize the density matrix

    # Compute the density matrix by summing over individual states
    for j in range(num_states):
        for f in range(num_states):
            # Contribution of state |ψj⟩ to the density matrix element ρf[j, f]
            rho_f[j, f] = psi[j] * np.conjugate(psi[f])  # j and f correspond to the row and column indices of the density matrix

    return rho_f

# Santi's Derivation Example

# Define the state vector psi
psi = [(1/2)*(0+1), (1/2)*(0-1)]  # [(1/2)(|0⟩ + |1⟩), (1/2)(|0⟩ - |1⟩)]


# Compute the density matrix
rho_f = compute_density_matrix(psi)

# Print the density matrix with real values
print(np.real(rho_f))


In [ ]:
def compute_density_matrix(psi):
    rho = np.kron(psi, psi.conjugate().T)
    return rho

#### Here we compute the collapsed state of the qubit after measurment error

In [ ]:
def compute_density_matrix(psi):
    psi = np.array(psi)
    psi_conj = np.conjugate(psi)
    num_states = len(psi)  # Number of states in the superposition
    
    rho_f = np.outer(psi, psi_conj)  # Outer product of psi and its conjugate
    
    return rho_f

def compute_collapsed_state(psi, p_M):
    rho = compute_density_matrix(psi)
    rho_prime = (1-p_M)*rho + p_M*np.dot(sigma_x, np.dot(rho, sigma_x))
    
    trF0_rho, trF1_rho, trA0_rho_prime, trA1_rho_prime = trace_rho(rho, rho_prime)
    
    print("Expected Probability Measurement Values:")
    print("Probability value of measuring |0> with trF0_rho: ", trF0_rho)
    print("Probability value of measuring |1> with trF1_rho: ", trF1_rho)
    print("Probability value of measuring |0> with trA0_rho_prime after mapping : ", trA0_rho_prime)
    print("Probability value of measuring |1> with trA1_rho_prime after mapping: ", trA1_rho_prime)
    
    # Define the M_0 matrix
    M_0 = np.sqrt(1 - p_M) * np.kron(ket_0, bra_0) + np.sqrt(p_M) * np.kron(np.array(ket_1), bra_1)

    # Define the M_1 matrix
    M_1 = np.sqrt(1 - p_M) * np.kron(ket_1, bra_1) + np.sqrt(p_M) * np.kron(ket_0, bra_0)
    
    # Compute the collapsed state for outcome 0
    collapsed_state_0 = np.dot(np.dot(M_0, rho), M_0.conj().T)
    collapsed_state_0 /= np.trace(collapsed_state_0)  # Normalize the collapsed density matrix

    # Compute the collapsed state for outcome 1
    collapsed_state_1 = np.dot(np.dot(M_1, rho), M_1.conj().T)
    collapsed_state_1 /= np.trace(collapsed_state_1)  # Normalize the collapsed density matrix

    # Collapsed state for outcome 0
    print("\nCollapsed density matrix 0:")
    print(collapsed_state_0)
    
    final_state_0 = np.array([np.sqrt(collapsed_state_0[0][0]), np.sqrt(collapsed_state_0[1][1])])
    print("Final State 0: ", final_state_0[0], '|0> + ', final_state_0[1], '|1>')

    # Collapsed state for outcome 1
    print("\nCollapsed density matrix 1:")
    print(collapsed_state_1)
    
    final_state_1 = np.array([np.sqrt(collapsed_state_1[0][0]), np.sqrt(collapsed_state_1[1][1])])
    print("Final State 1: ", final_state_1[0], '|0> + ', final_state_1[1], '|1>')

# Example usage
# psi = 1/np.sqrt(2) * np.array([[1,1]])
psi = np.array([[.8, .6]])
p_M = 0.459889



compute_collapsed_state(psi, p_M)


## c. Extra Information <a id='extra'></a>

## Qubit Loss

Qubit loss refers to when a qubit is no longer accessible or present in a quantum system. This can be decoherence or physical degradation, loss through measurment, or coupling failure. When a qubit is lost, it can't be directly measured or coupled to other ancillary systems.

### Modeling Qubit Loss

Modeled by removing the lost qubit from the system. Reduces the dimensionality of the qubit space by a factor of two. Again, qubit loss is represented by tracing out the lost qubit, denoted as Tri(ρ). Where i = index of the lost qubit.

### Equivalence to Incoherent Processes

Qubit loss is equivalent to incoherent processes such as environmental coupling. Even though this model of qubit loss is equivalent to incoherent processes such as environmental coupling, correcting this type of error requires additional machinery on top of standard QEC protocols.

### Additional Machinery for Correcting Qubit Loss

As long as the physical object still exists in the system, Standard QEC protocols are designed to protect against the loss of information. Correcting qubit loss requires additional mechanisms. Without performing a projective measurement on the computational state, non-demolition detection protocols are used to determine if a qubit is present.

### Non-Demolition Detection

Without disturbing its state, Non-Demolition Detection Protocols help detect presence/absence of a qubit. Triggers additional actions or error correction procedures,  after detection of qubit loss, or absence of the qubit as indicated by the non-demolition detection protocol. These additonal actions are steps to replace the lost qubit or trigger the more advanced error correction techniques to mitigate the impact of the loss. 

 ### Standard QEC Protocols for Qubit Loss

As long as the physical qubit exists in the system. Standard QEC protocols protect against the loss of information on a qubit . Various QEC codes can be employed to address different types of errors.

### Standard QEC Protocol examples

Bit-Flip Code. Encoding: Error Encodes a logical qubit into three physical qubits using the following mappings: |0⟩L = |000⟩ and |1⟩L = |111⟩. Error Detection: Uses ancillary qubits used for syndrome measurement to detect bit-flip errors. Correction: Flipping the identified qubit to reverse the bit-flip error. 

Phase-Flip Code. Encoding: Same as bit-flip code Error Detection: Used syndrome measurement for phase-flip error. Correction: Adjusting the phase of the identified qubit to compensate for the phase-flip error

## Leakage 

Sources of Leakage

1. Internal fluctuations in the system that change the expected qubit dynamics can lead to leakage. This can occur when the qubit is improperly controlled, causing unintended population of additional levels in the system.  
2. Imprecise fabrication of the qubit system can also introduce leakage. Even in an otherwise perfect qubit system, fabrication imperfections can result in leakage into unintended levels. Such as, $U |0\rangle = \alpha |0\rangle + \beta |1\rangle + \gamma |2\rangle$ . Where  $\gamma |2\rangle$  represents the contribution of the leaked state $\vert2\rangle$ by improper control and , $U\vert0\rangle$ represents the resulting state after applying the operator on $\vert0\rangle$

### Leakage Correction

First a Non-Demolition, then a Standard QEC protocol. Error syndrome measurements will identify the specific qubits affected by the leakage. Internal fluctuations in the system and corrective operations can be performed to restore the logical qubit state.